
# **Extracting and Visualizing Stock Data using Python**

# Problem - 6: Plot GameStop Stock Graph


This comprehensive Python script showcases the integration of financial data extraction, cleaning, and visualization. Using yfinance, it retrieves GameStop (GME) stock data, while BeautifulSoup is employed to scrape its revenue data from an HTML source. The data is meticulously cleaned, ensuring datetime and numeric consistency, followed by the creation of interactive plots using Plotly's subplots. The custom make_graph function elegantly combines historical stock prices and revenue trends, delivering insightful visualizations tailored for financial analysis and decision-making.

In [2]:
!pip install yfinance
!pip install bs4
!pip install nbformat
!pip install --upgrade plotly

import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Define the make_graph function
def make_graph(stock_data, revenue_data, stock_name):
    # Remove timezone information from the Date columns
    stock_data['Date'] = stock_data['Date'].dt.tz_localize(None)
    revenue_data['Date'] = revenue_data['Date'].dt.tz_localize(None)

    # Filter data up to June 2021
    stock_data_specific = stock_data[stock_data['Date'] <= pd.to_datetime('2021-06-30')]
    revenue_data_specific = revenue_data[revenue_data['Date'] <= pd.to_datetime('2021-06-30')]

    # Create subplots for stock price and revenue
    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True,
        subplot_titles=("Historical Share Price", "Historical Revenue"),
        vertical_spacing=0.3
    )

    # Plot stock price
    fig.add_trace(
        go.Scatter(
            x=stock_data_specific['Date'], y=stock_data_specific['Close'],
            name="Share Price"
        ),
        row=1, col=1
    )

    # Plot revenue
    fig.add_trace(
        go.Scatter(
            x=revenue_data_specific['Date'], y=revenue_data_specific['Revenue'],
            name="Revenue"
        ),
        row=2, col=1
    )

    # Update axes and layout
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(
        title=stock_name, height=900, showlegend=False, xaxis_rangeslider_visible=True
    )
    fig.show()

# Step 1: Extract GME Stock Data using yfinance
game_stock_ticker = yf.Ticker("GME")
gme_data = game_stock_ticker.history(period="max")
gme_data.reset_index(inplace=True)
gme_data.head()

# Step 2: Web scrape Tesla Revenue Data
gme_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data2 = requests.get(gme_url).text
gme_soup = BeautifulSoup(html_data2,"html.parser")
table = gme_soup.find("table")
gme_revenue = pd.read_html(str(table))[0]
# Step 3: Clean Tesla Revenue Data
gme_revenue.columns = ['Date', 'Revenue']

gme_revenue['Date'] = pd.to_datetime(gme_revenue['Date'], errors='coerce')
gme_revenue['Revenue'] = gme_revenue['Revenue'].str.replace(',', '', regex=True).str.replace('$', '', regex=True)
gme_revenue['Revenue'] = pd.to_numeric(gme_revenue['Revenue'], errors='coerce')
gme_revenue.dropna(inplace=True)

# Step 4: Plot the data
make_graph(gme_data, gme_revenue, "GameStop Stock Price and Revenue")